# Imports

In [1]:
import cv2
import numpy as np
from sklearn.metrics import pairwise

c:\Python310\lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Global Variables

In [2]:
background = None

accumulated_weight = 0.5

roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600

In [3]:
def calcAccumulatedAvg(frame, accumulated_weight):
    global background

    if background is None:
        background = frame.copy().astype('float')
        return None
    
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [4]:
def segment(frame, threshold=25):
    global background
    
    diff = cv2.absdiff(background.astype('uint8'), frame)

    ret, thresholdedImg = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

    contours, hierarchy = cv2.findContours(thresholdedImg.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None
    
    else:
        handSegment = max(contours, key=cv2.contourArea)

        return (thresholdedImg, handSegment)

In [5]:
def countFingers(thresholdedImg, handSegment):

    convHull = cv2.convexHull(handSegment)
    top    = tuple(convHull[convHull[:, :, 1].argmin()][0])
    bottom = tuple(convHull[convHull[:, :, 1].argmax()][0])
    left   = tuple(convHull[convHull[:, :, 0].argmin()][0])
    right  = tuple(convHull[convHull[:, :, 0].argmax()][0])

    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2

    distance = pairwise.euclidean_distances([(cX, cY)], Y = [left, right, top, bottom])[0]

    maxDistance = distance.max()

    radius = int(0.8 * maxDistance)
    circumference = (2*np.pi*radius)

    circularRoi = np.zeros(thresholdedImg.shape[:2], dtype='uint8')

    cv2.circle(circularRoi, (cX, cY), radius, 255, 10)

    circularRoi = cv2.bitwise_and(thresholdedImg, thresholdedImg, mask=circularRoi)

    contours, hierarchy = cv2.findContours(circularRoi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    count = 0

    for contour in contours:

        (x,y,w,h) = cv2.boundingRect(contour)

        outOfWrist = ((cY + (cY * 0.25)) > (y + h))

        limitPoints = ((circumference * 0.25) > contour.shape[0])

        if outOfWrist and limitPoints:
            count += 1

    return count 


In [6]:
cam = cv2.VideoCapture(0)

numFrames = 0

while True:

    ret, frame = cam.read()
    frame = cv2.flip(frame, 1)
    frameCopy = frame.copy()

    roi = frame[roi_top: roi_bottom, roi_right:roi_left]

    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    gray = cv2.GaussianBlur(gray, (7,7), 0)

    if numFrames < 60:

        calcAccumulatedAvg(gray, accumulated_weight)

        if numFrames <= 59:

            cv2.putText(frameCopy, 'WAIT. GETTING BACKGROUND', (200,300), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow('finger count', frameCopy)
            
    else:

        hand = segment(gray)

        if hand is not None:

            thresholdedImg, handSegment = hand

            cv2.drawContours(frameCopy, [handSegment+(roi_right, roi_top)], -1, (255, 0, 0), 5)

            fingers = countFingers(thresholdedImg, handSegment)

            cv2.putText(frameCopy, str(fingers), (70, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255),2 )

            cv2.imshow('thresholded', thresholdedImg)


    cv2.rectangle(frameCopy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)

    numFrames += 1

    cv2.imshow('finger count', frameCopy)

    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()
